In [1]:
import requests
import json
import forecastio
from geopy.geocoders import Nominatim
import pandas as pd
import import_ipynb
from SFO_airbnb import SFO_airbnb
import datetime as dt
from config import api_key

importing Jupyter notebook from SFO_airbnb.ipynb


SFO_airbnb.ipynb:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  {
SFO_airbnb.ipynb:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  {
SFO_airbnb.ipynb:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  "outputs": [],


In [2]:
geolocator = Nominatim(user_agent="ETL_Project")

location = geolocator.geocode("San Francisco")

SFO_coord = []
SFO_coord.append(location.latitude)
SFO_coord.append(location.longitude)

SFO_coord

[37.7792808, -122.4192363]

In [3]:
forecast = forecastio.load_forecast(api_key, SFO_coord[0], SFO_coord[1])

byDay = forecast.daily()
print(byDay.summary)
print(byDay.icon)
print(byDay.data[4].temperatureHigh)

Light rain on Monday, with high temperatures peaking at 58°F on Thursday.
rain
58.19


In [4]:
temp_high = []
temp_low = []
precip_perc = []
precip_type = []
precip_int = []
date = []

for i in byDay.data:
    date.append(i.time.date())
    temp_high.append(i.temperatureHigh)
    temp_low.append(i.temperatureLow)
    precip_perc.append(i.precipProbability)
    precip_int.append(i.precipIntensity)
    try:
        precip_type.append(i.precipType)
    except:
        precip_type.append("none")

date

dates = [dt.datetime.strftime(i, '%Y-%m-%d') for i in date]
dates

['2018-12-22',
 '2018-12-23',
 '2018-12-24',
 '2018-12-25',
 '2018-12-26',
 '2018-12-27',
 '2018-12-28',
 '2018-12-29']

In [5]:
df = pd.DataFrame({"Date": dates, 
                     "High Temp": temp_high, 
                     "Low Temp": temp_low, 
                     "Precipitation Probability": precip_perc,
                     "Precipitation Intensity": precip_int,
                     "Precipitation Type": precip_type
                    })
df.dtypes

Date                          object
High Temp                    float64
Low Temp                     float64
Precipitation Probability    float64
Precipitation Intensity      float64
Precipitation Type            object
dtype: object

In [6]:
SFO_airbnb.dtypes

Date         object
Under $50     int64
$50-100       int64
$100-150      int64
$150-200      int64
$200-250      int64
dtype: object

In [8]:
SFO_table = pd.merge(df, SFO_airbnb, on="Date")
SFO_table["City"] = "San Francisco"
SFO_table

,Date,High Temp,Low Temp,Precipitation Probability,Precipitation Intensity,Precipitation Type,Under $50,$50-100,$100-150,$150-200,$200-250,City
0,2018-12-22,56.92,48.53,0.10,0.0003,rain,178,778,751,466,243,San Francisco
1,2018-12-23,57.27,50.08,0.24,0.0006,rain,192,792,751,457,255,San Francisco
2,2018-12-24,55.90,49.69,0.51,0.0075,rain,182,774,721,450,241,San Francisco
3,2018-12-25,57.87,48.74,0.32,0.0008,rain,188,763,724,463,241,San Francisco
4,2018-12-26,58.19,45.58,0.08,0.0001,rain,191,779,755,477,235,San Francisco
5,2018-12-27,58.37,47.49,0.00,0.0000,none,195,774,761,493,244,San Francisco
6,2018-12-28,55.80,43.58,0.00,0.0000,none,191,720,744,466,244,San Francisco
7,2018-12-29,57.68,48.32,0.00,0.0000,none,195,700,741,455,235,San Francisco
